In [1]:
with open("Sell_Short_Trades.txt") as fi:
    lines = fi.readlines()
    
lines[:60]

['                                                                                                                \n',
 ' DOTC                                                                                 RUN DATE : 06/14/17 21:56 \n',
 '                                                                                             RPT DATE :06/14/17 \n',
 '                                    TTS0126:SELL SHORT TRADES & SHORT EXEMPT                                    \n',
 ' Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr \n',
 '                                                                                                                \n',
 ' TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr      \n',
 ' CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        \n',
 '                                              

In [2]:
len(lines)

4158

# Start removing junk lines

In [3]:
# remove empty or whitespace-character-only strings
not_empty = [l for l in lines if l.strip()]
len(not_empty)

3735

In [4]:
# check a simple condition for page numbers
[l for l in lines if "of 77" in l]

['                                                       1 of 77                                                  \n',
 '                                                       2 of 77                                                  \n',
 '                                                       3 of 77                                                  \n',
 '                                                       4 of 77                                                  \n',
 '                                                       5 of 77                                                  \n',
 '                                                       6 of 77                                                  \n',
 '                                                       7 of 77                                                  \n',
 '                                                       8 of 77                                                  \n',
 '                                              

In [5]:
# remove the page number lines
pgs_removed = [l for l in not_empty if not "of 77" in l]
pgs_removed[:20]

[' DOTC                                                                                 RUN DATE : 06/14/17 21:56 \n',
 '                                                                                             RPT DATE :06/14/17 \n',
 '                                    TTS0126:SELL SHORT TRADES & SHORT EXEMPT                                    \n',
 ' Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr \n',
 ' TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr      \n',
 ' CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        \n',
 '                                                                    /17  /17  256945       00                   \n',
 ' 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO                                             \n',
 ' CERS     SS      NEW          100   2.3600001

In [6]:
no_rpt = pgs_removed[3:-2] # throw away report info at top and bottom
no_rpt[:10] # check top
no_rpt[-5:] # check bottom

['                                             854300                                                             \n',
 ' CERS     SS      NEW          786   2.350000    2.340000   2.35000006/1406/191706149900006 15:59:CustSS        \n',
 '                                                                    /17  /17  844088       59                   \n',
 ' 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS                                             \n',
 '                                             854300                                                             \n']

In [7]:
# extract and save column headers - not a bad thing to have for output
cols_lines = no_rpt[:2]
cols_str = ",".join([",".join(line.split()) for line in cols_lines])
cols_items = cols_str.split(",")

cols_str

'Symbol,Side,Cxl,Qty,Price,Bid,Ask,T-DatS-DatTradeID,TradeTiSS,Exbkr,TradeCommType,SourceCommission,Account,OrderID,GTL,Trailer,Info,Clr'

In [8]:
# Check if any column header lines don't match the style of the first instance
[x for x in no_rpt if "Symbol" in x and not x == cols_lines[0]]
[x for x in no_rpt if "TradeCommType" in x and not x == cols_lines[1]]

[]

In [9]:
headers_removed = [l for l in no_rpt if not l in cols_lines]
headers_removed[:20]

[' CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        \n',
 '                                                                    /17  /17  256945       00                   \n',
 ' 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO                                             \n',
 ' CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX  \n',
 '                                                                    /17  /17  866087       04                   \n',
 ' 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS                                             \n',
 '                                             854300                                                             \n',
 ' CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX  \n',
 '                                              

# Develop function for processing file

In [10]:
# Testing 
headers_removed[0].strip().startswith('CERS')

True

In [11]:
def get_chunk(lines: list) -> list:
    """Takes list of all lines, returns list containing the first group of lines"""
    group = []
    group_started = False
    counter = 0
    for line in lines:
        counter += 1
        l = line.strip()
        if (l.startswith('CERS') or l.startswith("S")): # first line of record
            if group_started:
                return group
            else:
                group.append(l)
                group_started = True
        else:
            group.append(l)
        if len(lines) == counter:
            return group

# Test function on a couple chunks' worth of data

In [12]:
h = headers_removed[:7] # first two records worth
h

[' CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS        \n',
 '                                                                    /17  /17  256945       00                   \n',
 ' 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO                                             \n',
 ' CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX  \n',
 '                                                                    /17  /17  866087       04                   \n',
 ' 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS                                             \n',
 '                                             854300                                                             \n']

In [13]:
[l.strip() for l in h]

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
 '/17  /17  256945       00',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300']

In [14]:
c = get_chunk(h)
print(len(c))
s2 = h[len(c):]
c2 = get_chunk(s2)

3


In [15]:
c2

['CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300']

# Actually process content of file

In [16]:
h = headers_removed
grouped_lines = []
while h:
    chunk = get_chunk(h)
    grouped_lines.append(chunk)
    h = h[len(chunk):] # number of lines of chunk may vary

In [17]:
grouped_lines

[['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
  '/17  /17  256945       00',
  '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO'],
 ['CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
  '/17  /17  866087       04',
  '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
  '854300'],
 ['CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
  '/17  /17  866096       04',
  '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
  '854300'],
 ['CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX',
  '/17  /17  869780       23',
  '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
  '854300'],
 ['CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 

# Prepare, then write output

In [18]:
def comma_join(str_list: list) -> str:
    "Takes a list of strings, splits them on whitespace, joins everything together with commas"
    return ",".join([",".join(s.split()) for s in str_list])

In [19]:
data_lines = [comma_join(l) for l in grouped_lines]
data_lines

['CERS,SS,NEW,2,756,2.400000,2.340000,2.45000006/1406/191706149900003,09:30:CustSS,/17,/17,256945,00,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=GSCO',
 'CERS,SS,NEW,100,2.36000018422.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866087,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,854300',
 'CERS,SS,NEW,200,2.350000,2.360000,2.37000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,866096,04,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,854300',
 'CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:20:ContraSSFREX,/17,/17,869780,23,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,854300',
 'CERS,SS,NEW,100,2.350000,2.350000,2.36000006/1406/191706149900003,10:22:ContraSSFREX,/17,/17,892888,29,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CERS,854300',
 'CERS,SS,NEW,100,2.360000,2.350000,2.36000006/1406/191706149900003,10:24:ContraSSFREX,/17,/17,909119,01,0704,PerUnit,TERM,0.010000,AABBC,1706149900003#STA=CE

In [20]:
with open('part2.csv', "w") as outfile:
    outfile.write(cols_str + "\n")
    all_text = "\n".join(data_lines)
    outfile.write(all_text)

# Alternate (better) approach
Note: csv wants an iterable - a list of items to write, and it will handle the comma-separating itself. Passing it a string will result in each character being written out separately.

In [21]:
# join lines together into one string, then split it into the elements - simpler than splitting then adding together
data_elems = [" ".join(l).split() for l in grouped_lines]
data_elems

[['CERS',
  'SS',
  'NEW',
  '2,756',
  '2.400000',
  '2.340000',
  '2.45000006/1406/191706149900003',
  '09:30:CustSS',
  '/17',
  '/17',
  '256945',
  '00',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003#STA=GSCO'],
 ['CERS',
  'SS',
  'NEW',
  '100',
  '2.36000018422.360000',
  '2.37000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '866087',
  '04',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003#STA=CERS',
  '854300'],
 ['CERS',
  'SS',
  'NEW',
  '200',
  '2.350000',
  '2.360000',
  '2.37000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '866096',
  '04',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '1706149900003#STA=CERS',
  '854300'],
 ['CERS',
  'SS',
  'NEW',
  '100',
  '2.350000',
  '2.350000',
  '2.36000006/1406/191706149900003',
  '10:20:ContraSSFREX',
  '/17',
  '/17',
  '869780',
  '23',
  '0704',
  'PerUnit',
  'TERM',
  '0.010000',
  'AABBC',
  '17061499

In [22]:
import csv

with open("part2_alt.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(cols_items)
    writer.writerows(data_elems)

Note: when the "record separator" value is not present in all lines (i.e with this file), pandas read_csv function raises a ParserError as it expects another field.